<a href="https://colab.research.google.com/github/JayKarhade/Breath-Classification/blob/main/Breath_detection_CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
import sklearn
import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Convolution1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense
from keras.regularizers import l2
from keras import backend as K
import pandas as pd
import numpy as np

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/breath_detect/dataset.csv')
df2 = pd.read_csv('/content/drive/MyDrive/breath_detect/class_labels.csv')

In [ ]:
x = df1.to_numpy()[:,:390]
y = df2.to_numpy()[:,1]
print(x.shape,y.shape)

(455, 390) (455,)


In [ ]:
#Shuffle data
indices = list(range(x.shape[0]))
np.random.shuffle(indices)
x = x[indices]
y=y[indices]
x = x.reshape(x.shape[0],x.shape[1],1)

In [ ]:
##Train-Test split
from sklearn.model_selection import train_test_split
y= to_categorical(y)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=2/8, random_state=1)

In [ ]:
input_cnn= (x_train.shape[1],1)

def model_1d_cnn():
  model = Sequential()
  model.add(Conv1D(filters=60, kernel_size=3, activation='relu',input_shape=input_cnn))
  model.add(BatchNormalization())
  #model.add(MaxPooling1D(pool_size=2))
  model.add(MaxPooling1D(pool_size=3, strides = 3))
  #model.add(BatchNormalization())
  model.add(Conv1D(filters=60, kernel_size=3, activation='relu'))
  #model.add(Dropout(0.7))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=120, kernel_size=5,))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=120,kernel_size=5,activity_regularizer=l2(0.0001)))
  model.add(Flatten())
  model.add(Dense(2000, activation='relu'))
  model.add(BatchNormalization())
  #model.add(Dropout(0.09))
  model.add(Dense(1000, activation='relu',activity_regularizer=l2(0.0001)))
  model.add(Dense(500, activation='relu',activity_regularizer=l2(0.0001)))
  #model.add(Dropout(0.15))
  model.add(Dense(100,activation='relu'))
  model.add(Dense(4,activation='softmax'))

In [ ]:
input_rnn = (x_train.shape[1],1)
model = Sequential()
model.add(LSTM(500, input_shape=input_rnn,return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(100,return_sequences=False))
model.add(Dense(1000))
model.add(BatchNormalization())
model.add(Dense(500))
model.add(Dropout(0.4))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(4)) 
print(model.summary)
#Set loss function
opt = keras.optimizers.Adam(lr=0.00005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
##Training and fitting the model
model.fit(x_train, y_train,epochs=30,verbose=1)
K.clear_session()

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f0d32885390>>
Epoch 1/30
8/8 [==============================] - 40s 4s/step - loss: 4.1129 - accuracy: 0.2419
Epoch 2/30
8/8 [==============================] - 36s 5s/step - loss: 1.4752 - accuracy: 0.2097
Epoch 3/30
8/8 [==============================] - 37s 5s/step - loss: 1.3707 - accuracy: 0.1577
Epoch 4/30
8/8 [==============================] - 36s 4s/step - loss: 1.3721 - accuracy: 0.1822
Epoch 5/30
8/8 [==============================] - 36s 4s/step - loss: 1.3871 - accuracy: 0.2224
Epoch 6/30
2/8 [======>.......................] - ETA: 26s - loss: 1.3873 - accuracy: 0.2656

KeyboardInterrupt: ignored